# DP1 target fields summary

The notebook produces table 2 -- Summary of the DP1 fields

In [1]:
# Generic python packages
import os
import csv
import requests
import yaml
import pandas as pd

# LSST Science Pipelines 
from lsst.daf.butler import Butler

# Set a standard figure size to use
from lsst.utils.plotting import publication_plots, get_multiband_plot_colors
import matplotlib.pyplot as plt

In [2]:
# Setup publication style
publication_plots.set_rubin_plotstyle()
colors = get_multiband_plot_colors()
bands = colors.keys()  # important to get the right order for plot legends
bands_dict = publication_plots.get_band_dicts()
%matplotlib inline

Set up Rubin matplotlib plot style.
This includes dicts for colors (bandpass colors for white background),
  colors_black (bandpass colors for black background), symbols, and line_styles,
  keyed on band (ugrizy).


In [3]:
# Custom formatter: no trailing zeros if not needed
def custom_float(x):
    if isinstance(x, float):
        s = f"{x:.6f}".rstrip('0').rstrip('.')
        return s
    return x

In [4]:
# Set up Butler
instrument = 'LSSTComCam'
collections = ['LSSTComCam/DP1/defaults', 
               'LSSTComCam/runs/DRP/DP1/v29_0_0/DM-50260',
               'skymaps', ]
skymap = 'lsst_cells_v1'
butler = Butler("/repo/dp1",
                instrument=instrument, 
                collections=collections, 
                skymap=skymap)
registry = butler.registry
skymap = butler.get('skyMap', skymap=skymap)

In [5]:
# Extract filter and pointing information about the fileds from the DP1 exposures
exposures = registry.queryDimensionRecords('exposure')
exp_df = pd.DataFrame(columns=['id', 'target', 'physical_filter','ra', 'dec'])
for count, info in enumerate(exposures):
    try:
        exp_df.loc[count] = [info.id, info.target_name, info.physical_filter, 
                         info.tracking_ra, info.tracking_dec]
    except: 
        print(">>>   Unexpected error:", sys.exc_info()[0])

In [6]:
# Physical filter -> band
exp_df['band'] = exp_df['physical_filter'].str.split('_').str[0]

In [7]:
# List of fields in DP1
dp1_fields = exp_df.target.unique()

In [8]:
# Load the FBS yaml file with target pointing centers
# This file has been copied to this repo from 
# https://github.com/lsst-ts/ts_config_ocs/blob/develop/Scheduler/feature_scheduler/maintel/fieldsurvey_centers.yaml
file_path = '../data/field_survey_centers.yaml'

try:
    # Open the file and load the YAML data
    with open(file_path, 'r') as file:
        targets = yaml.safe_load(file)

        if 'comcam_sv_targets' in targets:
            comcam_sv_targets = targets['comcam_sv_targets']

except FileNotFoundError:
    print(f"Error: The file at {file_path} was not found.")
except yaml.YAMLError as e:
    print(f"Error parsing the YAML file: {e}")

In [9]:
# Extract the FBS pointing centers for the fields in DP1
# Slew ICRS is not in the list
dp1_fields_and_centers = {field: pointing_center for field, pointing_center in comcam_sv_targets.items() if field in dp1_fields}
dp1_fields_and_centers

{'Rubin_SV_095_-25': {'description': 'Rubin SV Low Galactic Latitude Field',
  'ra': 95.0,
  'dec': -25.0},
 'ECDFS': {'description': 'Extended Chandra Deep Field South',
  'ra': 53.125,
  'dec': -28.1},
 'EDFS_comcam': {'description': 'Rubin SV Euclid Deep Field South',
  'ra': 59.1004,
  'dec': -48.73},
 'Rubin_SV_38_7': {'description': 'Rubin SV Low Ecliptic Latitude Field',
  'ra': 37.86,
  'dec': 6.98},
 'Fornax_dSph': {'description': 'Fornax Dwarf Spheroidal Galaxy',
  'ra': 39.9971,
  'dec': -34.4492},
 '47_Tuc': {'description': '47 Tucanae Globular Cluster',
  'ra': 6.022329,
  'dec': -72.081444},
 'Seagull': {'description': 'Seagull Nebula', 'ra': 106.23, 'dec': -10.51}}

In [10]:
# Statistics on the pointings 
stats = exp_df.groupby('target').size()
stats

# Group by and calculate various statistics
field_stats = exp_df.groupby('target')[['ra','dec']].agg([
    'count', 'mean', 'median', 'std', 'min',   'max',    

])
field_stats

ra                                                \
                 count        mean      median       std         min   
target                                                                 
47_Tuc              72    6.092413    6.127738  0.321065    5.564036   
ECDFS              719   53.146297   53.161735  0.111808   52.920257   
EDFS_comcam        272   59.125739   59.149250  0.147111   58.826435   
Fornax_dSph         42   40.063379   40.082025  0.119591   39.839461   
Rubin_SV_095_-25   292   95.026085   95.037502  0.107574   94.807038   
Rubin_SV_38_7      159   37.877536   37.976035  0.354119   37.368687   
Seagull            100  106.262080  106.266897  0.098514  106.088085   
slew_icrs          136   53.144349   53.165303  0.126223   52.907794   

                               dec                                             \
                         max count       mean     median       std        min   
target                                                                          
47_Tuc              6.619884    72 -72.095739 -72.090283  0.084791 -72.256177   
ECDFS              53.347572   719 -28.113223 -28.102455  0.095335 -28.290542   
EDFS_comcam        59.384158   272 -48.740920 -48.732493  0.096423 -48.920612   
Fornax_dSph        40.220448    42 -34.449456 -34.451064  0.071127 -34.623949   
Rubin_SV_095_-25   95.206899   292 -25.009301 -25.001371  0.097688 -25.190569   
Rubin_SV_38_7      38.395156   159   6.979206   7.015498  0.364579   6.394347   
Seagull           106.417737   100 -10.520814 -10.512528  0.095136 -10.700645   
slew_icrs          53.327818   136 -28.074607 -28.068646  0.089534 -28.275338   

                             
                        max  
target                       
47_Tuc           -71.922438  
ECDFS            -27.902422  
EDFS_comcam      -48.532337  
Fornax_dSph      -34.290400  
Rubin_SV_095_-25 -24.811694  
Rubin_SV_38_7      7.487474  
Seagull          -10.321652  
slew_icrs        -27.918140

In [11]:
# Add slew_icrs to the list of DP1 fields, use the median ra, dec for all science-grade visits included 
# slew_icrs_ra, slew_icrs_dec = exp_df[exp_df['target'] == 'slew_icrs'].groupby('target')[['ra', 'dec']].median().iloc[0].to_list()
slew_icrs_ra = field_stats.loc['slew_icrs', 'ra']['median']
slew_icrs_ra
slew_icrs_dec = field_stats.loc['slew_icrs', 'dec']['median']
dp1_fields_and_centers['slew_icrs'] = {'description': 'Engineering pointing',
                                       'ra': f"{slew_icrs_ra:.4g}", 'dec':f"{slew_icrs_dec:.4g}"}

{'Rubin_SV_095_-25': {'description': 'Rubin SV Low Galactic Latitude Field',
  'ra': 95.0,
  'dec': -25.0},
 'ECDFS': {'description': 'Extended Chandra Deep Field South',
  'ra': 53.125,
  'dec': -28.1},
 'EDFS_comcam': {'description': 'Rubin SV Euclid Deep Field South',
  'ra': 59.1004,
  'dec': -48.73},
 'Rubin_SV_38_7': {'description': 'Rubin SV Low Ecliptic Latitude Field',
  'ra': 37.86,
  'dec': 6.98},
 'Fornax_dSph': {'description': 'Fornax Dwarf Spheroidal Galaxy',
  'ra': 39.9971,
  'dec': -34.4492},
 '47_Tuc': {'description': '47 Tucanae Globular Cluster',
  'ra': 6.022329,
  'dec': -72.081444},
 'Seagull': {'description': 'Seagull Nebula', 'ra': 106.23, 'dec': -10.51},
 'slew_icrs': {'description': 'Engineering pointing',
  'ra': '53.17',
  'dec': '-28.07'}}

In [12]:
# Add in a description for the paper table 
# Download from github and update adding field descriptions

In [13]:
df_fields_summary = pd.DataFrame.from_dict(dp1_fields_and_centers, orient='index')
df_fields_summary.reset_index(inplace=True)
df_fields_summary.rename(columns={'index': 'field'}, inplace=True)
df_fields_summary= df_fields_summary.sort_values(by='field')
df_fields_summary

,field,description,ra,dec
5,47_Tuc,47 Tucanae Globular Cluster,6.022329,-72.081444
1,ECDFS,Extended Chandra Deep Field South,53.125,-28.1
2,EDFS_comcam,Rubin SV Euclid Deep Field South,59.1004,-48.73
4,Fornax_dSph,Fornax Dwarf Spheroidal Galaxy,39.9971,-34.4492
0,Rubin_SV_095_-25,Rubin SV Low Galactic Latitude Field,95.0,-25.0
3,Rubin_SV_38_7,Rubin SV Low Ecliptic Latitude Field,37.86,6.98
6,Seagull,Seagull Nebula,106.23,-10.51
7,slew_icrs,Engineering pointing,53.17,-28.07


In [14]:
exp_summary_table = table = exp_df.groupby(['target', 'band']).size().unstack(fill_value=0)[bands]
exp_summary_table

band,u,g,r,i,z,y
target,,,,,,
47_Tuc,6,10,32,19,0,5
ECDFS,25,212,206,121,125,30
EDFS_comcam,20,61,87,42,42,20
Fornax_dSph,0,5,25,12,0,0
Rubin_SV_095_-25,33,82,84,23,60,10
Rubin_SV_38_7,0,44,40,55,20,0
Seagull,10,37,43,0,10,0
slew_icrs,18,18,31,41,28,0


In [15]:
# Join on field/target 
result = pd.merge(df_fields_summary, exp_summary_table, how='left', left_on='field', right_on='target')
result

,field,description,ra,dec,u,g,r,i,z,y
0,47_Tuc,47 Tucanae Globular Cluster,6.022329,-72.081444,6,10,32,19,0,5
1,ECDFS,Extended Chandra Deep Field South,53.125,-28.1,25,212,206,121,125,30
2,EDFS_comcam,Rubin SV Euclid Deep Field South,59.1004,-48.73,20,61,87,42,42,20
3,Fornax_dSph,Fornax Dwarf Spheroidal Galaxy,39.9971,-34.4492,0,5,25,12,0,0
4,Rubin_SV_095_-25,Rubin SV Low Galactic Latitude Field,95.0,-25.0,33,82,84,23,60,10
5,Rubin_SV_38_7,Rubin SV Low Ecliptic Latitude Field,37.86,6.98,0,44,40,55,20,0
6,Seagull,Seagull Nebula,106.23,-10.51,10,37,43,0,10,0
7,slew_icrs,Engineering pointing,53.17,-28.07,18,18,31,41,28,0


In [16]:
# Format the date to produce a latex table
#Insert blank colum for nice spacing in table 
result.insert(4, ' ', ' ')

# Escape the underscores in the field names and codes
result.columns = result.columns.str.replace('_', r'\_', regex=False)
result = result.map(lambda x: x.replace('_', r'\_') if isinstance(x, str) else x)

data_latex = result.to_latex(index=False, escape=False, bold_rows=False,
                            formatters={col: custom_float for col in result.columns}
                            )

# Remove unnecessary latex
res = data_latex.split("midrule\n", 1)[-1]  # Keeps the part after 'midrule'
res = res.split("\\bottomrule", 1)[0]  # Keeps the part before 'bottomrule'

In [18]:
# Export to latex with deulxetable formatting 

with open("../tables/dp1_fields1.tex", "w") as f:
    f.write(r"""%%%%% This table is auto generated from data, DO NOT EDIT
\begin{deluxetable}{llcccp{0.5cm}p{0.5cm}p{0.5cm}p{0.5cm}p{0.5cm}p{0.5cm}}
\caption{DP1 fields and pointing centers with the number of images in each band per field.  
ICRS coordinates are in units of decimal degrees. 
\textcolor{red}{N IMAGES to be updated when data processing complete} \label{tab:dp1_fields} }
\tablehead{
  \colhead{\textbf{Field Code}} & \colhead{\textbf{Field Name}} & \colhead{\textbf{RA}} & \colhead{\textbf{DEC}} 
  & & \multicolumn{6}{c}{\textbf{Band}}\\
  \cline{3-4} \cline{6-11} 
  & & \colhead{deg}  & \colhead{deg}  & & u & g & r & i & z & y 
}
\startdata
""")
    f.write(res)
    f.write(r"""\enddata
\end{deluxetable}
""")
f.close()

# Plot of target fields 